In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ultralytics import YOLO
from pathlib import Path
import sys
import os
import cv2
import numpy as np

ROOT = Path().resolve()
SRC_DIR = ROOT / "src"

if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from utils_LF import yolo_late_fusion, draw_fused_boxes

In [3]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["ULTRALYTICS_IGNORE_MULTIPROCESS"] = "1"

In [5]:
model_rgb = YOLO("runs/detect/wildlife_rgb_v8n_cpu5/weights/best.pt")
model_t   = YOLO("runs/detect/wildlife_t_v8n_cpu/weights/best.pt")

print("Modelos cargados:")
print("  RGB -> runs/detect/wildlife_rgb_v8n_cpu/weights/best.pt")
print("  T   -> runs/detect/wildlife_t_from_rgb_v8n_cpu/weights/best.pt")

VAL_RGB_DIR = "data/format_rgb/images/val"
VAL_T_DIR   = "data/format_t/images/val"

OUT_DIR = "runs/late_fusion/val"
os.makedirs(OUT_DIR, exist_ok=True)

print("Directorio de salida:", OUT_DIR)


Modelos cargados:
  RGB -> runs/detect/wildlife_rgb_v8n_cpu/weights/best.pt
  T   -> runs/detect/wildlife_t_from_rgb_v8n_cpu/weights/best.pt
Directorio de salida: runs/late_fusion/val


In [6]:
class_names = {
    0: "Cow",
    1: "Deer",
    2: "Horse",

}

class_names


{0: 'Cow', 1: 'Deer', 2: 'Horse'}

In [7]:
val_rgb_path = Path(VAL_RGB_DIR)
img_paths_rgb = sorted(
    list(val_rgb_path.glob("*.jpg")) +
    list(val_rgb_path.glob("*.png"))
)

print(f"Encontradas {len(img_paths_rgb)} imágenes RGB en {VAL_RGB_DIR}.")

for img_rgb_path in img_paths_rgb:
    img_name = img_rgb_path.name

    img_t_path = Path(VAL_T_DIR) / img_name

    if not img_t_path.exists():
        print(f"[WARN] No se encontró térmica para {img_name}, se saltea.")
        continue

    res_rgb_list = model_rgb(
        source=str(img_rgb_path),
        imgsz=614,
        device="cpu",
        verbose=False
    )
    res_rgb = res_rgb_list[0]

    res_t_list = model_t(
        source=str(img_t_path),
        imgsz=614,
        device="cpu",
        verbose=False
    )
    res_t = res_t_list[0]

    fused = yolo_late_fusion(
        res_rgb,
        res_t,
        iou_det=0.6,   # umbral IoU detección (paper)
        prob_thr=0.45, # umbral probabilidad (paper)
        iou_match=0.3  # mínimo IoU para considerar que dos cajas "matchean"
    )

    img_bgr = cv2.imread(str(img_rgb_path))
    img_out = draw_fused_boxes(img_bgr, fused, class_names=class_names)

    out_path = os.path.join(OUT_DIR, img_name)
    cv2.imwrite(out_path, img_out)

    print(f"[OK] Guardado: {out_path}")

print("Listo: imágenes con detecciones Late Fusion guardadas en:", OUT_DIR)


Encontradas 32 imágenes RGB en data/format_rgb/images/val.
[WARN] No se encontró térmica para 020221_deer_pens_xt2_DJI_0048.JPG, se saltea.
[WARN] No se encontró térmica para 020221_deer_pens_xt2_DJI_0360.JPG, se saltea.
[WARN] No se encontró térmica para 022521_DJI_0132.JPG, se saltea.
[WARN] No se encontró térmica para 060421_DJI_0676.JPG, se saltea.
[WARN] No se encontró térmica para 060421_DJI_0678.JPG, se saltea.
[WARN] No se encontró térmica para 071521_DeerPens_xt2_DJI_0070.JPG, se saltea.
[WARN] No se encontró térmica para 071521_DeerPens_xt2_DJI_0132.JPG, se saltea.
[WARN] No se encontró térmica para 071521_DeerPens_xt2_DJI_0398.JPG, se saltea.
[WARN] No se encontró térmica para 071521_DeerPens_xt2_DJI_0464.JPG, se saltea.
[WARN] No se encontró térmica para 071521_DeerPens_xt2_DJI_0480.JPG, se saltea.
[WARN] No se encontró térmica para CA_DJI_0488.jpg, se saltea.
[WARN] No se encontró térmica para CA_DJI_0502.jpg, se saltea.
[WARN] No se encontró térmica para CA_DJI_0528.jpg, 